In [1]:
from google.colab import drive
drive.mount('./drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at ./drive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = F.relu(self.i2h(combined))
        output = self.i2o(combined)
        return output, hidden
    
    def initHidden(self, batch_size):
        return torch.zeros(batch_size, self.hidden_size)

In [0]:
def make_id_dic(fname):
    id_dic = {}
    
    with open(fname) as file:
        for i, line in enumerate(file, 1):
            line = line.rstrip('\n')
            id_dic[line] = i
            
    return id_dic

In [0]:
def words2id(sentence, id_dic):
    import re
    import snowballstemmer
    
    words_id = []
    
    # 文字種の統一
    sentence = sentence.lower()
    
    # 数字の置き換え
    sentence = re.sub(r'[0-9]+', '0', sentence)
    
    # '-'を' 'に変換
    sentence = sentence.replace('-', ' ')
    
    words = sentence.split()
    
    # ステミング処理
    stemmer = snowballstemmer.stemmer('english')
    words2 = [stemmer.stemWord(word) for word in words]
    words = words2
    
    for word in words:
        if word in id_dic.keys():
            words_id.append(id_dic[word])
        else:
            words_id.append(0)
            
    return words_id

In [0]:
def id2vec(words_id, embeds):
    words_vec = embeds(torch.Tensor(words_id).long())
    
    return words_vec

In [0]:
def make_dataset(fname, id_dic, embeds):
    with open(fname) as file:
        lines = file.readlines()
        lines_vec = []
        labels = np.zeros([len(lines), 1])
        
        for i, line in enumerate(lines):
            
            line = line.rstrip('\n')
            category = line.strip('\t')[0]
            title = line.split('\t')[1]
            
            # lines_vecの処理
            words_id = words2id(title, id_dic)
            words_vec = id2vec(words_id, embeds)
            lines_vec.append(words_vec)
            
            # labelsの処理
            if category == 'b':
                labels[i] = 0
            elif category == 't':
                labels[i] = 1
            elif category == 'e':
                labels[i] = 2
            elif category == 'm':
                labels[i] = 3
    labels = torch.from_numpy(labels).long()
                
    return lines_vec, labels

In [0]:
def train(rnn, criterion, optimizer, batch_size, words_vec_batch, label_batch):
    hidden = rnn.initHidden(batch_size)
    rnn.zero_grad()
    optimizer.zero_grad()
    hidden = hidden.detach()
    
    for i in range(len(words_vec_batch)):
        output, hidden = rnn(words_vec_batch[i].to(device), hidden.to(device))
    loss = criterion(output, label_batch.to(device))
    loss.backward(retain_graph=True)
    optimizer.step()

In [0]:
def get_loss_and_accyracy(rnn, criterion, lines_vec, labels):
    running_loss = 0
    correct_count = 0
    
    for words_vec, label in zip(lines_vec, labels):
        words_vec_batch, label_batch = batch_process([words_vec], [label])
        hidden = rnn.initHidden(1)
        
        for i in range(len(words_vec_batch)):
            output, hidden = rnn(words_vec_batch[i].to(device), hidden.to(device))
        loss = criterion(output, label.to(device))
        running_loss += loss
        pre_label = torch.max(output, 1)[1]
        if pre_label == label.to(device):
            correct_count += 1
            
    return running_loss / len(labels), correct_count / len(labels)

In [0]:
def batch_process(words_vec_list, label_list):
    # words_vec_batchの処理
    len_max = 0
    for words_vec in words_vec_list:
        if words_vec.size()[0] > len_max:
            len_max = words_vec.size()[0]
    words_vec_batch = torch.zeros(len_max, len(words_vec_list), 300)
    for i in range(len(words_vec_list)):
        for j in range(len(words_vec_list[i])):
            words_vec_batch[len_max - len(words_vec_list[i]) + j, i, :] = words_vec_list[i][j]
        
    # label_batchの処理
    label_batch = torch.zeros(len(label_list)).long()
    for i, label in enumerate(label_list):
        label_batch[i] = label
        
    return words_vec_batch, label_batch

In [17]:
import numpy as np
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('{0}\n'.format(device))

rnn = RNN(300, 50, 4).to(device) # RNN(d_w, d_h, L)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(rnn.parameters(), lr=pow(10, -2)) # lr: 検証の結果、10エポック目で最も精度が高かった数値

id_dic = make_id_dic('drive/My Drive/Colab Notebooks/100_knocks/chapter_6/train.feature.txt')
embeds = nn.Embedding(len(id_dic)+1, 300)
lines_vec, labels = make_dataset('drive/My Drive/Colab Notebooks/100_knocks/chapter_6/train.txt', id_dic, embeds)
lines_vec_test, labels_test = make_dataset('drive/My Drive/Colab Notebooks/100_knocks/chapter_6/test.txt', id_dic, embeds)
batch_size = 8

for epoch in range(10):
    words_vec_list = []
    label_list = []
    
    for i, dataset in enumerate(zip(lines_vec, labels), 1):
        words_vec_list.append(dataset[0])
        label_list.append(dataset[1])
        
        if i % batch_size == 0 or i == len(lines_vec):
            words_vec_batch, label_batch = batch_process(words_vec_list, label_list)
            train(rnn, criterion, optimizer, len(words_vec_list), words_vec_batch, label_batch)
            words_vec_list = []
            label_list = []
            
    loss_train, accuracy_train = get_loss_and_accyracy(rnn, criterion, lines_vec, labels)
    loss_test, accuracy_test = get_loss_and_accyracy(rnn, criterion, lines_vec_test, labels_test)
    print('[{0}] 訓練データ上の損失: {1}\t訓練データ上の正解率: {2}\t評価データ上の損失: {3}\t評価データ上の正解率: {4}'.format(epoch + 1, loss_train, accuracy_train, loss_test, accuracy_test))

cuda:0

[1] 訓練データ上の損失: 0.9523890018463135	訓練データ上の正解率: 0.6196405167571616	評価データ上の損失: 1.054468035697937	評価データ上の正解率: 0.5884557721139431
[2] 訓練データ上の損失: 0.821818470954895	訓練データ上の正解率: 0.6797416214192099	評価データ上の損失: 0.9685491919517517	評価データ上の正解率: 0.6356821589205397
[3] 訓練データ上の損失: 0.6597511172294617	訓練データ上の正解率: 0.7570679648005991	評価データ上の損失: 0.8244378566741943	評価データ上の正解率: 0.7076461769115442
[4] 訓練データ上の損失: 0.5194805264472961	訓練データ上の正解率: 0.8061224489795918	評価データ上の損失: 0.7122222781181335	評価データ上の正解率: 0.7586206896551724
[5] 訓練データ上の損失: 0.45064449310302734	訓練データ上の正解率: 0.8348623853211009	評価データ上の損失: 0.6739144325256348	評価データ上の正解率: 0.767616191904048
[6] 訓練データ上の損失: 0.39617297053337097	訓練データ上の正解率: 0.8559258565811646	評価データ上の損失: 0.6396747827529907	評価データ上の正解率: 0.7841079460269865
[7] 訓練データ上の損失: 0.3454422354698181	訓練データ上の正解率: 0.8737127878674406	評価データ上の損失: 0.6308897137641907	評価データ上の正解率: 0.7796101949025487
[8] 訓練データ上の損失: 0.3047354519367218	訓練データ上の正解率: 0.8893465643137989	評価データ上の損失: 0.6274599432945251	評価データ上の正解率: 0.78